In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/medicaltranscriptions/mtsamples.csv")
print(df.columns)

In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd
import os, sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from os.path import join
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn

import torch

In [ ]:
import pandas as pd
import os, sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from os.path import join
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn

import torch

print(os.listdir())

datapath = "data"

cuda_available = torch.cuda.is_available()

def eval_metrics(actual, pred):
    bal_acc = balanced_accuracy_score(actual,pred)
    f1_sc = f1_score(actual,pred,average="micro")
    return bal_acc, f1_sc


def data_loader(filename):
    df = pd.read_csv(join(datapath,filename))
    df.drop(['Unnamed: 0'],axis=1,inplace=True)
    
    
    counts = df['medical_specialty'].value_counts()
    others = [k for k,v in counts.items() if v<100]
    for each_spec in others:
        df.loc[df['medical_specialty']==each_spec,'medical_specialty']=' others' 
    
    counts = df['medical_specialty'].value_counts()
    print(counts)
    
    num_classes = len(df['medical_specialty'].unique())
    class_dict = dict(zip(df['medical_specialty'].unique(),list(range(num_classes))))
    df['medical_specialty'] = df['medical_specialty'].apply(lambda x:class_dict[x])
    df.dropna(inplace=True)
    df['transcription'] = df['keywords']+df['transcription'] 
    X = df[['transcription']]
    y = df[["medical_specialty"]]

    
    train_x,test_x,train_y,test_y = train_test_split(X,y, stratify=y,test_size=0.25)
    
    # class_weights = sklearn.utils.class_weight.compute_class_weight("balanced",list(set(list(y.values))),list(y.values))
    class_weights = [1]*num_classes
    
    print(df.head())
    
    return train_x, train_y, test_x, test_y, num_classes, class_weights, class_dict
    
    
if __name__ == '__main__':
    train_x,train_y,test_x,test_y, num_classes, class_weights, class_dict = data_loader("/kaggle/input/medicaltranscriptions/mtsamples.csv")
    
    train_df = pd.DataFrame(columns=['text','labels'])
    train_df['text'] = train_x['transcription']
    train_df['labels'] = train_y['medical_specialty']
    print(train_df.head())
    test_df = pd.DataFrame(columns=['text','labels'])
    test_df['text'] = test_x['transcription']
    test_df['labels'] = test_y['medical_specialty']
    

    learning_rate = 1e-5
    num_of_epochs = 3
        
    model_args = ClassificationArgs(num_train_epochs=num_of_epochs,learning_rate = learning_rate,  reprocess_input_data= True,save_model_every_epoch=False, overwrite_output_dir= True)
    
    model = ClassificationModel(
        "roberta",
        "roberta-base",
        num_labels=num_classes,
        weight=class_weights,
        use_cuda=cuda_available,
        args=model_args
        )
   
    model.train_model(train_df)
    model.save_model()
    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    

    
    
    result,output = model.predict(test_df['text'].values.tolist())
    

    
    acc, f1 = eval_metrics(test_df['labels'],result)

    print(acc,f1)
    
    

In [ ]:
test_df.to_csv("test_df.csv")

In [ ]:
saved_model = ClassificationModel(
    "roberta", "/kaggle/working/outputs"
)

In [ ]:
result,output = saved_model.predict(test_df['text'].values.tolist())
acc, f1 = eval_metrics(test_df['labels'],result)
print(acc,f1)